In [1]:
import pandas as pd

In [2]:
diffusion_stats = pd.read_csv('/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/scripts/tmp/diffusion_statistics_2025-08-21T00:32:20.341372.csv', index_col = 0)
singan_stats = pd.read_csv('/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/scripts/tmp/singan_statistics_2025-08-21T00:34:35.481989.csv', index_col = 0)

In [5]:
diffusion_stats['cv_subject'].value_counts()

cv_subject
FD-031    2081
FD-027    2078
FD-030    1973
FD-029    1903
FD-032    1885
Name: count, dtype: int64

In [6]:
singan_stats['cv_subject'].value_counts()

cv_subject
FD-029    1800
FD-027    1800
FD-031    1800
FD-032    1800
FD-030    1800
Name: count, dtype: int64

In [7]:
sample_size = 1500

In [ ]:
import pandas as pd

# collect one record per subject per iteration
records = []

for i in range(100):
    for cv_subject in diffusion_stats['cv_subject'].unique():
        df_sub = (
            diffusion_stats
            .loc[diffusion_stats['cv_subject'] == cv_subject]
            .sample(sample_size, random_state=i)
        )
        records.append({
            'cv_subject':        cv_subject,
            'iteration':         i,
            'mean_ssim':         df_sub['ssim'].mean(),
            'median_ssim':       df_sub['ssim'].median(),
            'ssim_pct>0.2':      (df_sub['ssim'] > 0.2).mean(),
            'ssim_pct>0.5':      (df_sub['ssim'] > 0.5).mean(),
            'mean_psnr':         df_sub['psnr'].mean(),
            'median_psnr':       df_sub['psnr'].median(),
            'kl_mean_div':       df_sub['dist_align'].mean(),
            'kl_median_div':     df_sub['dist_align'].median(),
            'sifid_mean':        df_sub['sifid'].mean(),
            'sifid_median':      df_sub['sifid'].median(),
            'map_mean':          df_sub['map'].mean(),
            'map_median':        df_sub['map'].median(),
            'miou_mean':         df_sub['miou'].mean(),
            'miou_median':       df_sub['miou'].median(),
            'pct_cohesive':      df_sub['is_cohesive_mask_image'].mean(),
        })

# DataFrame of all 100×#subjects runs
df_runs = pd.DataFrame.from_records(records)

# Now aggregate: for each subject, mean of each metric over the 100 runs
df_agg = (
    df_runs
    .groupby('cv_subject')
    .mean()
    .reset_index()
).drop('iteration', axis = 1)

# df_agg will have columns:
#    cv_subject, mean_ssim, median_ssim, ssim_pct>0.2, …, pct_cohesive


metrics = [
    'mean_ssim', 'median_ssim', 'ssim_pct>0.2', 'ssim_pct>0.5',
    'mean_psnr', 'median_psnr',
    'kl_mean_div', 'kl_median_div',
    'sifid_mean', 'sifid_median',
    'map_mean', 'map_median',
    'miou_mean', 'miou_median',
    'pct_cohesive'
]

# 1) Compute per-subject means (you already have df_agg)
# 2) Compute overall means from df_runs
overall = df_runs[metrics].mean().to_frame().T
overall['cv_subject'] = 'All'

# 3) Make sure columns align
#    (move cv_subject to front)
cols = ['cv_subject'] + metrics
overall = overall[cols]

# 4) Concatenate
df_final = pd.concat([df_agg[cols], overall], ignore_index=True)

  cv_subject  iteration  mean_ssim  median_ssim  ssim_pct>0.2  ssim_pct>0.5  \
0     FD-027       49.5   0.182911     0.122084      0.148113      0.076400   
1     FD-029       49.5   0.202815     0.152302      0.182773      0.083387   
2     FD-030       49.5   0.235699     0.163488      0.274247      0.105113   
3     FD-031       49.5   0.166932     0.127911      0.092073      0.059020   
4     FD-032       49.5   0.192688     0.139494      0.181287      0.079953   

   mean_psnr  median_psnr  kl_mean_div  kl_median_div  sifid_mean  \
0  11.258731    11.033705     3.360672       0.916670  223.573721   
1  11.169385    10.678517     3.411172       1.065100  251.035685   
2  13.144723    10.808838     2.594528       1.384034  251.349905   
3  10.057185     9.155389     3.608910       1.473175  251.299781   
4   9.776837     7.831140     4.819534       3.346649  268.304223   

   sifid_median  map_mean  map_median  miou_mean  miou_median  pct_cohesive  
0    204.738037  0.057874    0.0

In [13]:
df_agg

,cv_subject,iteration,mean_ssim,median_ssim,ssim_pct>0.2,ssim_pct>0.5,mean_psnr,median_psnr,kl_mean_div,kl_median_div,sifid_mean,sifid_median,map_mean,map_median,miou_mean,miou_median,pct_cohesive
0,FD-027,49.5,0.182911,0.122084,0.148113,0.076400,11.258731,11.033705,3.360672,0.916670,223.573721,204.738037,0.057874,0.030477,0.040551,0.026569,0.642420
1,FD-029,49.5,0.202815,0.152302,0.182773,0.083387,11.169385,10.678517,3.411172,1.065100,251.035685,247.329580,0.098888,0.057554,0.079452,0.035617,0.568953
2,FD-030,49.5,0.235699,0.163488,0.274247,0.105113,13.144723,10.808838,2.594528,1.384034,251.349905,251.353955,0.074475,0.033803,0.055617,0.025177,0.404953
3,FD-031,49.5,0.166932,0.127911,0.092073,0.059020,10.057185,9.155389,3.608910,1.473175,251.299781,243.689169,0.068232,0.034485,0.051154,0.031121,0.611667
4,FD-032,49.5,0.192688,0.139494,0.181287,0.079953,9.776837,7.831140,4.819534,3.346649,268.304223,265.476003,0.053648,0.033795,0.035070,0.032339,0.469060


In [9]:
diffusion_stats_cv_subject

,cv_subject,key,styled_img_path,ssim,psnr,dist_align,sifid,map,mae,miou,is_cohesive_mask_image
8709,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.180628,12.481733,0.075300,256.423096,0.004436,0.049616,0.000000,True
1023,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.084583,8.355778,2.693965,285.693420,0.012268,0.888160,0.012268,True
3498,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.064488,4.874212,5.526945,420.395508,0.063019,0.834082,0.063019,True
3470,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.119555,3.650186,9.250743,344.908661,0.062622,0.848486,0.062622,True
9059,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.140578,11.890062,0.396454,223.191040,0.041756,0.089324,0.000000,True
...,...,...,...,...,...,...,...,...,...,...,...
1240,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.194119,3.030484,7.017141,287.000031,0.091348,0.072818,0.000000,False
6742,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.160077,3.111382,9.750638,436.626373,0.045990,0.142116,0.000000,False
5927,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.169284,11.419764,0.815616,197.465439,0.036977,0.153130,0.034312,True
9738,FD-029,/home/miguel/GI/1.5 - Synthetic Data Generatio...,/home/miguel/GI/1.5 - Synthetic Data Generatio...,0.160084,2.603169,8.192458,349.645142,0.068787,0.175060,0.000000,False
